In [4]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator

import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio

#### Parallel Function Calling Agent
GPT_VER = '1106' 
from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
parallel_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
    action_word = 'Tool invocations in parallel',
    observation_word = 'Tool-invocations results in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
    horizon = 5
)

query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
query_hello="Hello"
queries = [query, query_hello]

responses = await asyncio.gather( *[parallel_calling_agent._invoke_agent_action_async(q) for q in queries] ) 
responses



# parallel_calling_agent.clear_logs()
# parallel_calling_agent.run_agent_trials(num_trials=2, 
# query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
# reference="The former former president of Korea is older. The sum of their age is 99.")




[{'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_DAagYgst7cAZN81jiREMgZXR', function=Function(arguments='{"query": "former president of Korea"}', name='YDCSearch'), type='function'),
   ChatCompletionMessageToolCall(id='call_d65H5JWiKyYX4pGjQmLaALd0', function=Function(arguments='{"query": "MLB American League MVP 2022"}', name='YDCSearch'), type='function')]},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_IYKp4eH8NlfugQ6TfMhaZTCq', function=Function(arguments='{"query": "former president of South Korea"}', name='YDCSearch'), type='function'),
   ChatCompletionMessageToolCall(id='call_UFla1MxRdEhBfytkDDJQ5wXL', function=Function(arguments='{"query": "MLB American League MVP 2022"}', name='YDCSearch'), type='function')]},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [ChatCompletionMessageToolCall(id='call_fXiuPNqs9Hwj65KN93bB2Mbf', function=Function(arguments='{"query": "for

In [ ]:
client = QuickAzureOpenAIClient('1106')

In [ ]:
import asyncio
query="How are the current weathers in Seoul and Tokyo?"

await client.chat_completions_create_async(query)

In [5]:
from reasoning_engines.langchain_llm_wrappers import AsyncQuickAzureOpenAIClient, QuickAzureOpenAIClient
from utils.load_envs import Environ
env = Environ()
await env.openai_async()
message = [{
    'role':'assistant',
    'content':'You are a helpful assistant.'
},
{
    'role':'user',
    'content':'Who are the current weather in Seoul and New York?'
}]

from tools.get_tools import DefaultSchemasTools
from langchain.utils.openai_functions import convert_pydantic_to_openai_tool

schema_and_tool = DefaultSchemasTools('1106')
schemas = schema_and_tool.schemas()

openai_functions = [convert_pydantic_to_openai_tool(x) for x in schemas]


In [12]:
import asyncio
async_client = AsyncQuickAzureOpenAIClient('1106')
type(async_client)

# result = await asyncio.gather(*[async_client.chat_completions_create(messages=x, tools=openai_functions) for x in [message, message, message]])
# result
# await async_client.chat_completions_create(messages=message, tools=openai_functions)

reasoning_engines.langchain_llm_wrappers.AsyncQuickAzureOpenAIClient

In [6]:
client = QuickAzureOpenAIClient('1106')
result = [client.chat_completions_create(messages=x, tools=openai_functions) for x in [message, message, message]]
# client.chat_completions_create(messages=message, tools=openai_functions)

In [7]:
result

[ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_l5k9K4UK3jhlYzOHC8nlQbIB', function=Function(arguments='{"IANA_timezone": "Asia/Seoul"}', name='OpenWeatherMap'), type='function'), ChatCompletionMessageToolCall(id='call_s046XITbbygMQ8NMnuy0WBK7', function=Function(arguments='{"IANA_timezone": "America/New_York"}', name='OpenWeatherMap'), type='function')]),
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kLcrHl2spYKC5hknAxDQeNJ6', function=Function(arguments='{"IANA_timezone": "Asia/Seoul"}', name='OpenWeatherMap'), type='function'), ChatCompletionMessageToolCall(id='call_haONAvZ5vjbHZm1NezfjqBGg', function=Function(arguments='{"IANA_timezone": "America/New_York"}', name='OpenWeatherMap'), type='function')]),
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_

In [ ]:
import asyncio
import json

async def get_value(a):    
    return sum([a['a'], 100])

async def get_value_acync(a):
    result = sum([a, 100])
    return result

a = 100

await get_value_acync(a)

In [ ]:
import json
a = '{"a":3}'
json.loads(a)